<a href="https://colab.research.google.com/github/Suwandi-amin-sangaji/klasifikasi_sap/blob/main/Vgg16sapi1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Data / libarary

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
import os
import random
import numpy as np
from tensorflow import keras
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#Visualisasi Data

In [ ]:
def create_standard_directories():
    root_dir = '/content/drive/MyDrive/dataset/Sapi'
    subdirs = ['train/', 'val/']
    for subdir in subdirs:
        classes = ['Sapi Sehat', 'Sapi Sakit']
        for label_dir in classes:
            newdir = root_dir + subdir + label_dir
            os.makedirs(newdir, exist_ok=True)
            
create_standard_directories()

In [ ]:
from shutil import move
from tqdm import tqdm

def move_images_into_directories():
    train_dir = '/content/drive/MyDrive/dataset/Sapi'
    random.seed(1)
    validation_ratio = 0.25
    for img in tqdm(os.listdir(train_dir)):
        img_path = train_dir + img
        dst_dir = '/content/drive/MyDrive/dataset/sapi/train/'
        if random.random() < validation_ratio:
            dst_dir = '/content/drive/MyDrive/dataset/sapi/val/'
            
        if img.startswith('Sapi Sakit'):
            dst = dst_dir + 'Sapi Sakit/' + img
            move(img_path, dst)
        
        elif img.startswith('Sapi Sehat'):
            dst = dst_dir + 'Sapi Sehat/' + img
            move(img_path, dst)
        
        
move_images_into_directories()

#Model Vgg16

In [ ]:
from tensorflow.keras.applications import VGG16

vgg_base = VGG16(weights='imagenet', 
                 include_top=False, 
                 input_shape=(150, 150, 3)
                 )

vgg_base.summary()

In [ ]:
for layer in vgg_base.layers:
    layer.trainable = False
  
vgg_base.summary()

In [ ]:
model = Sequential([vgg_base,
                    Flatten(),
                    Dropout(0.3),
                    Dense(512, activation='relu'),
                    Dropout(0.2),
                    Dense(1024, activation='relu'),
                    Dropout(0.1),
                    Dense(1, activation='sigmoid')
                    ])
model.summary()

#MElakuakan Compile 


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#Augmentasi Data

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                             horizontal_flip=True,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             rotation_range=40,
                             shear_range=0.2,
                             fill_mode='nearest'
                                  )

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('/content/drive/MyDrive/dataset/Sapi/train',
                                              batch_size = 32,
                                              target_size = (150, 150),
                                              class_mode = 'binary'
                                              )

validation_generator = validation_datagen.flow_from_directory('/content/drive/MyDrive/dataset/Sapi/val',
                                                    batch_size = 32,
                                                    target_size = (150, 150),
                                                    class_mode = 'binary'
                                                    )

#Traning Data

In [ ]:
history = model.fit_generator(train_generator,
                    epochs=15,
                    validation_data=validation_generator
                   )

#Evaluasi Data


In [ ]:
#evaluasi model
score = model.evaluate(validation_generator)
print('Test Loss :',score[0])
print('Test Accuracy :',score[1])

#Plotting Data Traning Dan Validation


In [ ]:
#plotting training values
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
train_loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(train_acc) + 1)
plt.figure(figsize=(10,6))
plt.plot(epochs, train_acc, 'r', label = 'Training accuracy')
plt.plot(epochs, val_acc, 'b-*', label = 'Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.show()


plt.figure(figsize=(10,6))
plt.plot(epochs, train_loss, 'b-*', label = 'Training loss')
plt.plot(epochs, val_loss, 'r', label = 'Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

#Save Model

In [ ]:
model.save('/content/drive/MyDrive/dataset/Sapi/VGG_model.h5')

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, classification_report 

fig, ax = plt.subplots(figsize = (9, 6))

Y_pred = model.predict(train_generator)
y_pred = np.argmax(Y_pred, axis=1)
cm = confusion_matrix(train_generator.classes, y_pred)

disp = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = ["Sapi Sehat", "Sapi Sakit"])
disp.plot(cmap = plt.cm.Blues, ax = ax)

ax.set_title("Validation Set")
plt.show()

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
target_names = list(train_generator.class_indices.keys())
print(classification_report(train_generator.classes, y_pred, target_names=target_names))

#Testing 

In [ ]:
from tensorflow.keras.preprocessing import image
output_class = ['Sapi Sehat', 'Sapi Sakit']
def cow_prediction(new_image):
  test_image = image.load_img(new_image, target_size = (150,150))
  plt.axis("off")
  plt.imshow(test_image)
  plt.show()
 
  test_image = image.img_to_array(test_image) / 255
  test_image = np.expand_dims(test_image, axis=0)

  predicted_array = model.predict(test_image)
  predicted_value = output_class[np.argmax(predicted_array)]
  predicted_accuracy = round(np.max(predicted_array) * 100, 2)

  print(predicted_value, " with ", predicted_accuracy, " % accuracy")

In [ ]:
cow_prediction('/content/drive/MyDrive/dataset/Sapi/val/Sapi Sehat/img2434.jpg')